In [1]:
import os

from airtable import Airtable
import pandas as pd
import numpy as np
import seaborn as sns

from airtable_forms.common import config

# Make one dataframe with all CSV files

In [2]:
os.listdir('dictionaries')

['Brazil_2018_dictionary.csv',
 'EU_2019_dictionary.csv',
 'UK_2019_dictionary.csv',
 'UK_2017_dictionary.csv',
 'Germany_2017_dictionary.csv',
 'Mexico_2018_dictionary.csv',
 'Sweden_2018_dictionary.csv',
 'US_Midterm_2018_dictionary.csv']

In [12]:
def read_dict(fn):
    df = pd.read_csv('dictionaries/'+fn, keep_default_na=False, na_values=['_'])
    df['dictionary'] = fn.replace('_dictionary.csv','').replace('_', ' ')
    return df

dfs = [ read_dict(x) for x in os.listdir('dictionaries') ]
df = pd.concat(dfs)

# Quick cleanup

In [13]:
df[df['base_url'].str.contains(r'[^-a-z0-9.]')]

,base_url,code,dictionary
2512,trendsmap.com,SSA,EU 2019
1831,signon.thomsonreuters.com?productid=plcuk&view...,PN,UK 2017
51,block.fiverr.com?url=l3myl2mwmjy2nda2zmy,SSA,Sweden 2018
52,block.fiverr.com?url=l3mylzcxy2ewmjfin2y,SSA,Sweden 2018
97,block.fiverr.com?url=l2fzaglrbw9yztevyw1hem9ul...,SSA,US Midterm 2018
98,block.fiverr.com?url=l3mylzdlnziymgy4yza,SSA,US Midterm 2018
99,block.fiverr.com?url=l3mylzq0nwq1mznhmwq,SSA,US Midterm 2018
914,vote.gop?utm_source=twitter_gop&utm_medium=soc...,PP,US Midterm 2018


In [14]:
df = df[~df['base_url'].str.contains(r'[^-a-z0-9.]')]

In [15]:
df[pd.isna(df['code'])]

,base_url,code,dictionary


# Some descriptive stats

In [16]:
len(df)

10737

In [17]:
df['dictionary'].value_counts()

EU 2019            2796
UK 2017            2437
Germany 2017       1736
UK 2019            1235
US Midterm 2018     981
Brazil 2018         532
Mexico 2018         515
Sweden 2018         505
Name: dictionary, dtype: int64

In [18]:
df['code'].value_counts()

CC     1998
PN2    1278
JN      898
PP      775
PN      766
PN1     689
OTN     678
OP      608
L       557
PN3     512
G       348
SSA     290
EX      280
OTP     145
SM      130
SHC     122
RE       94
NA       76
PB       75
LI       66
FP       65
RU       64
PH       63
TB       63
LS       39
SF       26
CL       18
PM        8
WL        6
Name: code, dtype: int64

In [19]:
vc = df['base_url'].value_counts()
vc[vc>5]

instagram.com         9
nytimes.com           8
reuters.com           8
open.spotify.com      8
pscp.tv               8
change.org            8
bbc.com               8
msn.com               7
theguardian.com       7
spreaker.com          7
mailchi.mp            7
ft.com                7
pbs.twimg.com         7
amazon.com            7
google.com            7
mobile.twitter.com    7
twitter.com           7
paper.li              7
medium.com            7
washingtonpost.com    7
soundcloud.com        7
vimeo.com             7
en.m.wikipedia.org    6
bloomberg.com         6
zerohedge.com         6
flickr.com            6
wsj.com               6
sputniknews.com       6
linkedin.com          6
cnbc.com              6
itunes.apple.com      6
dailymail.co.uk       6
foxnews.com           6
m.facebook.com        6
gofundme.com          6
drive.google.com      6
independent.co.uk     6
breitbart.com         6
facebook.com          6
vice.com              6
youtube.com           6
apnews.com      

In [20]:
pd.DataFrame(vc.values)[0].value_counts()

1    7741
2     905
3     162
4      71
6      24
5      22
7      15
8       6
9       1
Name: 0, dtype: int64

In [21]:
len(df['base_url'].unique())

8947

# Import dictionaries (called "Projects" for now)

In [10]:
airtable = Airtable(config["base-key"], "Projects", api_key=config["api-key"])

In [15]:
dicts = df['dictionary'].unique()
dicts

array(['Brazil 2018', 'EU 2019', 'UK 2019', 'UK 2017', 'Germany 2017',
       'Mexico 2018', 'Sweden 2018', 'US Midterm 2018'], dtype=object)

In [16]:
for d in dicts:
    airtable.insert({'Name':d})

# Import categories

In [22]:
airtable = Airtable(config["base-key"], "Categories", api_key=config["api-key"])

In [23]:
df.groupby('code')['dictionary'].unique()

code
CC     [Brazil 2018, EU 2019, UK 2019, UK 2017, Germa...
CL     [Brazil 2018, EU 2019, UK 2019, Mexico 2018, S...
EX     [Brazil 2018, EU 2019, UK 2019, UK 2017, Germa...
FP     [Brazil 2018, EU 2019, UK 2019, Mexico 2018, S...
G      [Brazil 2018, EU 2019, UK 2019, UK 2017, Germa...
JN     [Brazil 2018, EU 2019, UK 2019, UK 2017, Germa...
L      [Brazil 2018, EU 2019, UK 2019, UK 2017, Germa...
LI     [Brazil 2018, EU 2019, UK 2019, Mexico 2018, S...
LS     [Brazil 2018, EU 2019, UK 2019, Mexico 2018, U...
NA     [Brazil 2018, Germany 2017, Mexico 2018, Swede...
OP     [Brazil 2018, EU 2019, UK 2019, UK 2017, Germa...
OTN    [Brazil 2018, EU 2019, UK 2019, UK 2017, Germa...
OTP    [Brazil 2018, EU 2019, UK 2019, Mexico 2018, S...
PB      [Brazil 2018, EU 2019, Mexico 2018, Sweden 2018]
PH     [EU 2019, UK 2019, UK 2017, Germany 2017, Mexi...
PM                               [UK 2017, Germany 2017]
PN     [Brazil 2018, UK 2017, Germany 2017, Mexico 20...
PN1    [Brazil 2018, EU 20

In [24]:
for code, dicts in df.groupby('code')['dictionary'].unique().items():
    airtable.insert({'Code': code, 'Project': list(dicts)}, typecast=True)

# Import sources (base_urls)

In [46]:
airtable = Airtable(config["base-key"], "Sources", api_key=config["api-key"])

In [66]:
df['base_url'].unique()

array(['abuladomercado.com.br', 'ac24horas.com', 'acoluna.co', ...,
       'yourperceptionisnotreality.wordpress.com', 'yourvoiceamerica.tv',
       'yvonneforcongress.com'], dtype=object)

In [67]:
for source in df['base_url'].unique():
    airtable.insert({'Base URL': source})

# Insert coding decisions

In [25]:
airtable = Airtable(config["base-key"], "Coding decisions", api_key=config["api-key"])

In [ ]:
for idx, row in df.iterrows():
    airtable.insert(
        {'Source': row['base_url'], 'Category': row['code'], 'Project': row['dictionary']},
        typecast=True
    )